In [1]:
class Vowels:
    def __getitem__(self, i):
        return 'AEIOU'[i]
    
v = Vowels()
v[0]

'A'

In [2]:
v[-1]

'U'

In [3]:
for c in v: print(c)

A
E
I
O
U


In [4]:
'E' in v

True

In [5]:
'Z' in v

False

In [1]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

In [2]:
#monkey patching
from random import shuffle
l = list (range(10))
shuffle(l)
l


[7, 9, 1, 2, 3, 4, 0, 6, 8, 5]

In [3]:
deck = FrenchDeck()
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

In [4]:
def set_card(deck, position,card):
    deck._cards[position] = card

FrenchDeck.__setitem__ = set_card
shuffle(deck)
deck[:5]

[Card(rank='10', suit='hearts'),
 Card(rank='6', suit='hearts'),
 Card(rank='10', suit='spades'),
 Card(rank='10', suit='clubs'),
 Card(rank='4', suit='clubs')]

In [7]:
#defensive programming and "fail fast"
def __init__(self, iterable):
    self._balls = list(iterable)

In [8]:
#Goose typing
from collections.abc import Sequence
Sequence.register(FrenchDeck)

__main__.FrenchDeck

In [9]:
#subclassing an ABC
from collections import namedtuple, abc

Card = namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                       for rank in self.ranks]
    
    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

    def __setitem__(self, position, value):  
        self._cards[position] = value

    def __delitem__(self, position):  
        del self._cards[position]

    def insert(self, position, value):  
        self._cards.insert(position, value)


In [10]:
import abc

class Tombola(abc.ABC):
    
    @abc.abstractmethod
    def load(self, iterable):
        """Add items from iterable"""
        
    @abc.abstractmethod
    def pick(self):
        """Remove item at random, returning it
        
        This method should raise LookUpError when the instance is empty
        """
    
    def loaded(self):
        """"Return True if there is at least 1 item."""
        return bool(self.inspect())
    
    def inspect(self):
        """return a sorted tuple with the items currently inside"""
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(items)
                

In [11]:
class Fake(Tombola):
    
    def pick(self):
        return 13

Fake
    

__main__.Fake

In [12]:
f = Fake()

TypeError: Can't instantiate abstract class Fake without an implementation for abstract method 'load'

In [13]:
#ABC Syntax details
class MyABC(abc.ABC):
    @classmethod
    @abc.abstractmethod #innermost decorator
    
    def an_abstract_class_method(cls,_):
        pass

In [15]:
#Subclassing an ABC

import random

class BingoCage(Tombola):
    
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
        
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('empty cage')
    
def __call__(self):
    self.pick()

In [16]:
class LottoBlower(Tombola):
    
    def __init__(self, iterable):
        self._balls = list(iterable)
        
    def load(self, iterable):
        self._balls.extend(iterable)
        
    def pick(self):
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from an empty blower')
        return self._balls.pop(position)
    
    def loaded(self):
        return bool (self._balls)
    
    def inspect(self):
        return tuple(self._balls)

In [17]:
#virtual subclass
from random import randrange

@Tombola.register
class TomboList(list):
    def pick(self):
        if self:  
            position = randrange(len(self))
            return self.pop(position)  
        else:
            raise LookupError('pop from empty TomboList')

    load = list.extend  

    def loaded(self):
        return bool(self)  

    def inspect(self):
        return tuple(self)


In [18]:
issubclass(TomboList, Tombola)

True

In [19]:
t = TomboList(range(100))

In [20]:
isinstance(t, Tombola)

True

In [21]:
TomboList.__mro__

(__main__.TomboList, list, object)

In [24]:
#structual typing with ABCs

class Struggle:
    
    def __len__(self): return 23
    

from collections import abc
isinstance(Struggle(), abc.Sized)

True

In [27]:
issubclass(Struggle, abc.Sized)

True

In [ ]:
class Sized(metaclass=ABCMeta):

    __slots__ = ()

    @abstractmethod
    def __len__(self):
        return 0

    @classmethod
    def __subclasshook__(cls, C):
        if cls is Sized:
            if any("__len__" in B.__dict__ for B in C.__mro__):  
                return True  
        return NotImplemented  

In [30]:
abc.Sized.__subclasshook__

<bound method Sized.__subclasshook__ of <class 'collections.abc.Sized'>>

In [33]:
abc.Sized.__subclasscheck__

<bound method ABCMeta.__subclasscheck__ of <class 'collections.abc.Sized'>>

In [35]:
# Static protocols 
# typed double function

def double(x):
    return x * 2

double(1.5)

3.0

In [36]:
double('A')

'AA'

In [37]:
double([1, 2, 3])

[1, 2, 3, 1, 2, 3]

In [38]:
from fractions import Fraction
double(Fraction(1, 2))

Fraction(1, 1)

In [40]:
from typing import TypeVar, Protocol

T = TypeVar('T')

class Repeatable(Protocol):
    def __mul__(self: T, repeat_count: int)->T:...
    
RT = TypeVar('RT', bound=Repeatable)

def double(x : RT) -> RT:
    return x * 2

In [ ]:
@runime_checkable
class SupportsComplex(Protocol):
    """An ABC with one abstract method __complex__."""
    
    __slots__ = ()
    
    @abstractmethod
    def __complex__(self) -> complex:
        pass